# Data Transformation and Loading
---
Transformation of:
- Minneapolis weather data, hourly readings
- Ogilvie solar panel data
- Minneapolis solar panel data

In [20]:
import pandas as pd
from sqlalchemy import create_engine

## Weather Data

In [21]:
# read in minneapolis weather data
weather_df = pd.read_csv("Resources/openweather_minneapolis_4_17-9_19.csv")

# display column names
# for column in weather_df.columns:
#     print(column)

In [22]:
# preview
weather_df.head()

,dt,dt_iso,city_id,city_name,lat,lon,temp,temp_min,temp_max,pressure,...,rain_today,snow_1h,snow_3h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1491004800,2017-04-01 00:00:00 +0000 UTC,5037649,NaN,NaN,NaN,286.86,286.15,287.15,1017,...,NaN,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03d
1,1491008400,2017-04-01 01:00:00 +0000 UTC,5037649,NaN,NaN,NaN,284.40,282.15,286.15,1018,...,NaN,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03n
2,1491012000,2017-04-01 02:00:00 +0000 UTC,5037649,NaN,NaN,NaN,281.85,278.15,285.15,1019,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
3,1491015600,2017-04-01 03:00:00 +0000 UTC,5037649,NaN,NaN,NaN,278.82,275.15,284.15,1019,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n
4,1491019200,2017-04-01 04:00:00 +0000 UTC,5037649,NaN,NaN,NaN,277.49,274.15,283.15,1020,...,NaN,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01n


In [23]:
# function to convert temp in kelvin to fahrenheit
def k_to_f(temp_k):
    temp_f = (temp_k - 273.15) * 9/5 + 32
    return temp_f

In [24]:
# function to limit 'dt_iso' to sql timestamp format
def get_timestamp(dt):
    
    # return only format: YYYY-MM-DD HH:MM:SS
    timestamp = dt[0:19]
    return timestamp

In [25]:
# set up blank lists for converted temperatures and datetimes
temp_f_list = []
timestamp_list = []

# convert datetime format, save into list
for dt in weather_df['dt_iso']:
    timestamp_list.append(get_timestamp(dt))

# convert kelvin to fahrenheit
for temp in weather_df['temp']:
    temp_f_list.append(k_to_f(temp))
    
# save lists into df
weather_df['temp_f'] = temp_f_list
weather_df['weather_date_time'] = timestamp_list

In [26]:
# useful/relevant data
relevant_cols = [
    'weather_date_time',
    'temp_f',
    'rain_1h',
    'snow_1h',
    'clouds_all',
    'weather_description',
    'weather_main',
    'pressure',
    'wind_speed',
    'wind_deg',
    'humidity'
]

# pair-down df to only relevant cols
final_weather_df = weather_df[relevant_cols].copy()

In [27]:
# fill NaN precip. values with '0.0'
final_weather_df.fillna(0.0, inplace=True)
final_weather_df.drop_duplicates(subset='weather_date_time', inplace=True)

# set weather_date_time as index
final_weather_df.set_index('weather_date_time', inplace=True)
final_weather_df.head()

,temp_f,rain_1h,snow_1h,clouds_all,weather_description,weather_main,pressure,wind_speed,wind_deg,humidity
weather_date_time,,,,,,,,,,
2017-04-01 00:00:00,56.678,0.0,0.0,40,scattered clouds,Clouds,1017,3,10,30
2017-04-01 01:00:00,52.250,0.0,0.0,40,scattered clouds,Clouds,1018,2,350,34
2017-04-01 02:00:00,47.660,0.0,0.0,1,sky is clear,Clear,1019,2,36,56
2017-04-01 03:00:00,42.206,0.0,0.0,1,sky is clear,Clear,1019,2,36,49
2017-04-01 04:00:00,39.812,0.0,0.0,1,sky is clear,Clear,1020,2,36,60


In [28]:
connection_string = "postgres:postgres@localhost:5432/solar_weather_db"
engine = create_engine(f'postgresql://{connection_string}')

# Confirm tables
engine.table_names()

['weather', 'solar_ogilvie', 'solar_minneapolis']

In [29]:
final_weather_df.to_sql(name='weather', con=engine, if_exists='append', index=True)
print("weather data added")

weather data added


In [30]:
# test for successful data addition
pd.read_sql('select * from weather', con=engine).head()

,weather_date_time,weather_description,clouds_all,temp_f,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_1h,weather_main
0,2017-04-01 00:00:00,scattered clouds,40,56.678,1017,30,3,10,0.0,0.0,Clouds
1,2017-04-01 01:00:00,scattered clouds,40,52.250,1018,34,2,350,0.0,0.0,Clouds
2,2017-04-01 02:00:00,sky is clear,1,47.660,1019,56,2,36,0.0,0.0,Clear
3,2017-04-01 03:00:00,sky is clear,1,42.206,1019,49,2,36,0.0,0.0,Clear
4,2017-04-01 04:00:00,sky is clear,1,39.812,1020,60,2,36,0.0,0.0,Clear


## Solar Data - Ogilvie Panel

In [12]:
# read solar panel data into df
olg_power_df = pd.read_csv("Resources/Ashley_Solar_Data.csv")
olg_power_df.head()

FileNotFoundError: [Errno 2] File b'Resources/Ashley_Solar_Data.csv' does not exist: b'Resources/Ashley_Solar_Data.csv'

In [13]:
# rename columns
solar_olg_transform = olg_power_df.rename(columns=\
                                          {"Date/Time": "og_date_time",
                                                        "Power Delivered (W)": "power_delivered",
                                                        "Energy Delivered (Wh)": "energy_delivered",
                                                        "Cumulative Energy Delivered (Wh)": "cumulative_energy"})

solar_olg_transform.head()

,og_date_time,power_delivered,energy_delivered,cumulative_energy
0,4/21/2019 11:30,1139.0,NaN,NaN
1,4/21/2019 11:45,14856.0,3714.0,4853.0
2,4/21/2019 12:00,14892.0,3723.0,8576.0
3,4/21/2019 12:15,15524.0,3881.0,12457.0
4,4/21/2019 12:30,11136.0,2784.0,15241.0


In [14]:
# clean data: drop null values (1st row & last row)
solar_olg_transform.dropna(how='any', inplace=True)
solar_olg_transform.head()

,og_date_time,power_delivered,energy_delivered,cumulative_energy
1,4/21/2019 11:45,14856.0,3714.0,4853.0
2,4/21/2019 12:00,14892.0,3723.0,8576.0
3,4/21/2019 12:15,15524.0,3881.0,12457.0
4,4/21/2019 12:30,11136.0,2784.0,15241.0
5,4/21/2019 12:45,12876.0,3219.0,18460.0


Format date_time

In [15]:
# prep datetimes by padding single-digit days, months, and hours with a '0'

# create list of og_date_time
dt_list = list(solar_olg_transform.og_date_time)

# iterate through indices of dt_list
for i in range(len(dt_list)):
    # make month format %m (from %-m)
    # if 2nd character is '/' then add a '0' at beginning
    if dt_list[i][1] == '/':
        dt_list[i] = f"0{dt_list[i]}"
    
    # make day format %d (from %-d)
    # if 4th char is '/' then pad day with '0'
    if dt_list[i][4] == '/':
        dt_list[i] = f"{dt_list[i][:3]}0{dt_list[i][3:]}"
    
    # make hours format %H (from %-H)
    # if 12th char is ':' then pad hours with a '0'
    if dt_list[i][12] == ':':
        dt_list[i] = f"{dt_list[i][:11]}0{dt_list[i][11:]}"

In [16]:
# function that accepts a datetime string from solar_olg_transform and reformats as YYYY-MM-DD HH:MM:SS
def format_solar_dt(dt):
    year = dt[6:10]
    month = dt[:2]
    day = dt[3:5]
    time = dt[11:]
    
    timestamp = f"{year}-{month}-{day} {time}:00"
    return timestamp

In [17]:
# blank list to be appended to
formatted_dt_list = []

# loop through list of prepped datetime strings
for dt in dt_list:
    
    # perform format function on each item in list
    formatted_dt_list.append(format_solar_dt(dt))

# add formatted_dt_list to df
solar_olg_transform['og_date_time'] = formatted_dt_list

solar_olg_transform.head()

,og_date_time,power_delivered,energy_delivered,cumulative_energy
1,2019-04-21 11:45:00,14856.0,3714.0,4853.0
2,2019-04-21 12:00:00,14892.0,3723.0,8576.0
3,2019-04-21 12:15:00,15524.0,3881.0,12457.0
4,2019-04-21 12:30:00,11136.0,2784.0,15241.0
5,2019-04-21 12:45:00,12876.0,3219.0,18460.0


In [18]:
# drop NaN values and duplicate rows
solar_olg_transform.dropna(inplace=True)
solar_olg_transform.drop_duplicates(inplace=True)

# set index as timestamp
solar_olg_transform.set_index('og_date_time', inplace=True)

solar_olg_transform.head()

,power_delivered,energy_delivered,cumulative_energy
og_date_time,,,
2019-04-21 11:45:00,14856.0,3714.0,4853.0
2019-04-21 12:00:00,14892.0,3723.0,8576.0
2019-04-21 12:15:00,15524.0,3881.0,12457.0
2019-04-21 12:30:00,11136.0,2784.0,15241.0
2019-04-21 12:45:00,12876.0,3219.0,18460.0


In [19]:
# insert df into postgresql db
solar_olg_transform.to_sql(name='solar_ogilvie', con=engine, if_exists='append', index=True)
print("olg solar data added")

olg solar data added


In [20]:
# test for successful addition into database
# perform a join just for fun!
pd.read_sql(
    'select * from solar_ogilvie '\
    'right join weather '\
    'on solar_ogilvie.og_date_time = weather.weather_date_time', con=engine)

,og_date_time,power_delivered,energy_delivered,cumulative_energy,weather_date_time,weather_description,clouds_all,temp_f,rain_1h,snow_1h
0,NaT,NaN,NaN,NaN,2017-04-01 00:00:00,scattered clouds,40,56.678,0,0
1,NaT,NaN,NaN,NaN,2017-04-01 01:00:00,scattered clouds,40,52.250,0,0
2,NaT,NaN,NaN,NaN,2017-04-01 02:00:00,sky is clear,1,47.660,0,0
3,NaT,NaN,NaN,NaN,2017-04-01 03:00:00,sky is clear,1,42.206,0,0
4,NaT,NaN,NaN,NaN,2017-04-01 04:00:00,sky is clear,1,39.812,0,0
5,NaT,NaN,NaN,NaN,2017-04-01 05:00:00,sky is clear,1,38.498,0,0
6,NaT,NaN,NaN,NaN,2017-04-01 06:00:00,sky is clear,1,35.348,0,0
7,NaT,NaN,NaN,NaN,2017-04-01 07:00:00,sky is clear,1,35.618,0,0
8,NaT,NaN,NaN,NaN,2017-04-01 08:00:00,sky is clear,1,34.412,0,0
9,NaT,NaN,NaN,NaN,2017-04-01 09:00:00,sky is clear,1,33.926,0,0


## Minneapolis Solar Panel

In [28]:
# load in data into df
mpls_df = pd.read_csv("Resources/Minneapolis_Solar_Data.csv")

# rename columns
mpls_df.columns = ['mpls_date_time', 'power_delivered']

# replace blank readings with '0.0'
mpls_df.fillna(0.0, inplace=True)
mpls_df.head()

,mpls_date_time,power_delivered
0,5/26/2017 14:45,445.0
1,5/26/2017 15:45,2280.0
2,5/26/2017 16:45,4186.0
3,5/26/2017 17:45,4283.0
4,5/26/2017 18:45,4043.0


In [22]:
# calculate energy dilivered and cumulative energy delivered
mpls_df['energy_delivered'] = 0.25 * mpls_df['power_delivered']
mpls_df['cumulative_energy'] = mpls_df.energy_delivered.cumsum()

mpls_df.head()

,mpls_date_time,power_delivered,energy_delivered,cumulative_energy
0,5/26/2017 14:45,445.0,111.25,111.25
1,5/26/2017 15:45,2280.0,570.00,681.25
2,5/26/2017 16:45,4186.0,1046.50,1727.75
3,5/26/2017 17:45,4283.0,1070.75,2798.50
4,5/26/2017 18:45,4043.0,1010.75,3809.25


In [23]:
# prep datetimes by padding single-digit days, months, and hours with a '0'

# create list of og_date_time
dt_list = list(mpls_df.mpls_date_time)

# iterate through indices of dt_list
for i in range(len(dt_list)):
    # make month format %m (from %-m)
    # if 2nd character is '/' then add a '0' at beginning
    if dt_list[i][1] == '/':
        dt_list[i] = f"0{dt_list[i]}"
    
    # make day format %d (from %-d)
    # if 4th char is '/' then pad day with '0'
    if dt_list[i][4] == '/':
        dt_list[i] = f"{dt_list[i][:3]}0{dt_list[i][3:]}"
    
    # make hours format %H (from %-H)
    # if 12th char is ':' then pad hours with a '0'
    if dt_list[i][12] == ':':
        dt_list[i] = f"{dt_list[i][:11]}0{dt_list[i][11:]}"

In [24]:
# blank list to be appended to
formatted_dt_list = []

# loop through list of prepped datetime strings
for dt in dt_list:
    
    # perform format function on each item in list
    formatted_dt_list.append(format_solar_dt(dt))

# create new df with transformed timestamp
mpls_transform = mpls_df.copy()
    
# add formatted_dt_list to df
mpls_transform['mpls_date_time'] = formatted_dt_list

mpls_transform.head()

,mpls_date_time,power_delivered,energy_delivered,cumulative_energy
0,2017-05-26 14:45:00,445.0,111.25,111.25
1,2017-05-26 15:45:00,2280.0,570.00,681.25
2,2017-05-26 16:45:00,4186.0,1046.50,1727.75
3,2017-05-26 17:45:00,4283.0,1070.75,2798.50
4,2017-05-26 18:45:00,4043.0,1010.75,3809.25


In [25]:
# drop_duplicate rows, set index as date_time
mpls_transform.drop_duplicates(inplace=True)
mpls_transform.set_index('mpls_date_time', inplace=True)

mpls_transform.head()

,power_delivered,energy_delivered,cumulative_energy
mpls_date_time,,,
2017-05-26 14:45:00,445.0,111.25,111.25
2017-05-26 15:45:00,2280.0,570.00,681.25
2017-05-26 16:45:00,4186.0,1046.50,1727.75
2017-05-26 17:45:00,4283.0,1070.75,2798.50
2017-05-26 18:45:00,4043.0,1010.75,3809.25


In [26]:
# insert df into postgresql db
mpls_transform.to_sql(name='solar_minneapolis', con=engine, if_exists='append', index=True)
print("mpls solar data added")

mpls solar data added


In [27]:
# test for successful insertion into postgresql db
pd.read_sql('select * from solar_minneapolis', con=engine).head()

,mpls_date_time,power_delivered,energy_delivered,cumulative_energy
0,2017-05-26 14:45:00,445,111,111
1,2017-05-26 15:45:00,2280,570,681
2,2017-05-26 16:45:00,4186,1047,1728
3,2017-05-26 17:45:00,4283,1071,2799
4,2017-05-26 18:45:00,4043,1011,3809
